# pCO2 validation

In [ ]:
chunk_start

In [ ]:
ff = "../../matched/gridded/pco2/pco2.nc"
ds_model = nc.open_data(ff)
mask_all(ds_model)
years = set(ds_model.years)
year_min = min(years)
year_max = max(years)
year_range = f"{year_min}-{year_max}"
ds_model.subset(variable = "model")
ds_model.tmean("month")
ds_model.as_missing(0)
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
ds_annual.set_longnames({"model": "surface pCO2"})

In [ ]:
ds_obs = nc.open_data(ff)
ds_obs.subset(variable = "observation")
ds_obs.run()
ds_obs.tmean("month")
ds_obs.as_missing(0)
ds_obs.run()

ds_annual_obs = ds_obs.copy()
ds_annual_obs.tmean()
ds_annual_obs.set_longnames({"observation": "Sea surface pCO2"})


In [ ]:
obs_mask = ds_obs.copy()
obs_mask > -1e20
mod_mask = ds_model.copy()
mod_mask > -1e20
mod_mask * obs_mask
mod_mask.run()
ds_model * mod_mask
ds_obs * mod_mask

In [ ]:
chunk_clim

In [ ]:
chunk_bias

The ability of the model to reproduce seasonality of pCO2 is assessed by comparing the modelled and observed seasonal cycle of pCO2. The seasonal cycle is calculated by averaging the monthly values of pCO2 over all available model years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle of pCO2. The observed seasonal cycle is calculated by averaging the observed monthly values of pCO2 over all available years. The seasonal cycle is calculated for each grid cell. The modelled seasonal cycle is compared to the observed seasonal cycle using the correlation coefficient between the two. The correlation coefficient is calculated for each grid cell. The correlation coefficient ranges from -1 to 1. A value of 1 indicates a perfect agreement between the modelled and observed seasonal cycle of pCO2. A value of -1 indicates a perfect disagreement between the modelled and observed seasonal cycle of pCO2. A value of 0 indicates no agreement between the modelled and observed seasonal cycle of pCO2. 


In [ ]:
ds1 = ds_model.copy()
ds1.cdo_command("setname,model")
ds1.run()
ds2 = ds_obs.copy()
ds2.cdo_command("setname,observation")
ds2.run()
ds_cor = nc.open_data([ds1.current[0], ds2.current[0]])
ds_cor.merge(match=["month"])
ds_cor.run()
ds_ts = ds_cor.copy()
ds_cor.cor_time("model", "observation")
title = f"Seasonal temporal correlation between {variable} for model and observations"
ds_cor.run()


# output to nc

out = f"../../results/temporals/{variable}_cor.nc"
if not os.path.exists(os.path.dirname(out)):
    os.makedirs(os.path.dirname(out))
ds_cor.to_nc(out, zip = True, overwrite = True)

In [ ]:

df_cor = ds_cor.to_dataframe().reset_index()
# get range of lon and lat without missing values of cor in df_cor
ds_cor.pub_plot()

In [ ]:
md(f"**Figure {i_figure}**: Seasonal temporal correlation between model and observations for " + variable)
i_figure += 1

## Can the model reproduce long-term changes in sea surface pCO2?

In [ ]:
md(f"The ability of the model to reproduce the observed long-term changes in {variable} is evaluated by calculating the linear trend of the time series. The trend is calculated for the years {year_range} using annual mean values.") 

In [ ]:
ds_lt = nc.open_data(ff)
mask_all(ds_lt)
ds_lt.tmean("year")
ds_lt.cor_time("model", "observation")
ds_lt.run()
ds_lt.pub_plot()

In [ ]:
md(f"**Figure {i_figure}**: The long-term temporal correlation between model and observations for surface pCO2. The correlation is calculated for the years {year_range} using annual mean values.") 
i_figure += 1

In [ ]:
md_result , i_figure = ecoval.global_regionals(ds_model, ds_obs, variable, i_figure)

In [ ]:
md_result

In [ ]:
ds_annual = ds_model.copy()
ds_annual.rename({ds_annual.variables[0]: "model"})
ds_annual.append(ds_obs)
ds_annual.tmean()
ds_annual.merge("variables")
ds_annual.rename({ds_obs.variables[0]: "observation"})
out_dir = "../../results/annual_mean/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + f"annualmean_{variable}.nc"
ds_annual.to_nc(out_file, zip = True, overwrite = True)

In [ ]:
chunk_end